# Matrix
## Create a matrix containing Mandelbrot set data

Create an image from a simple fractal
The Mandelbrot set is the set of complex numbers c for which the function
f(z) = z^2 + c does not diverge when iterated from z = 0.
The function is iterated until the absolute value of z exceeds 2 or a maximum
number of iterations is reached.

The Mandelbrot set is colored according to the number of iterations needed
to determine that a point is not in the set. Points in the set are colored black.
The color of a point outside the set is determined by the number of iterations
before the absolute value of z exceeds 2.

### Complex numbers

In [ ]:
// install SixLabors.ImageSharp
#r "nuget: SixLabors.ImageSharp"


In [2]:
using SixLabors.ImageSharp.PixelFormats;
using SixLabors.ImageSharp;

In [3]:
struct Complex
{
    public double Real { get; }
    public double Imaginary { get; }

    public Complex(double real, double imaginary)
    {
        Real = real;
        Imaginary = imaginary;
    }

    public static Complex operator +(Complex c1, Complex c2)
    {
        return new Complex(c1.Real + c2.Real, c1.Imaginary + c2.Imaginary);
    }

    public static Complex operator *(Complex c1, Complex c2)
    {
        return new Complex(c1.Real * c2.Real - c1.Imaginary * c2.Imaginary, c1.Real * c2.Imaginary + c1.Imaginary * c2.Real);
    }

    public double MagnitudeSquared()
    {
        return Real * Real + Imaginary * Imaginary;
    }
}

## translate value to color

In [4]:
Color GetColor(double mu)
{
    mu *= 150;
    int clr1 = (int)mu;
    double t2 = mu - clr1;
    double t1 = 1 - t2; 
    var colorCount = (int)Math.Pow(2, 24);
    clr1 = clr1 % colorCount;
    int clr2 = (clr1 + 1000) % colorCount;

    byte r = (byte)((clr1 >> 16) * t1 + (clr2 >> 16) * t2);
    byte g = (byte)((clr1>>8 & 0xff) * t1 + (clr2 >> 8 & 0xff) * t2);
    byte b = (byte)((clr1&0xff) * t1 + (clr2&0xff) * t2);
 
    return new Color (new Argb32(r,g,b,255));
}

### Create the fractal

In [ ]:
int width = 800;
int height = 800;
Image<Argb32> bitmap = new(width, height);

double xmin = -3.5;
double xmax = 2.5;
double ymin = -4.0;
double ymax = 4.0;

for (int x = 0; x < width; x++)
{
    for (int y = 0; y < height; y++)
    {
        double a = xmin + (xmax - xmin) * x / width;
        double b = ymin + (ymax - ymin) * y / height;

        Complex c = new Complex(a, b);
        Complex z = new Complex(0, 0);

        int iterations = 0;
        int maxIterations = 2000;

        while (iterations < maxIterations && z.MagnitudeSquared() < 4.0)
        {
            z = z * z + c;
            iterations++;
        }
        
        Color color = GetColor(iterations + 1 - 
            Math.Log(Math.Log(z.MagnitudeSquared())) / Math.Log(2));

        bitmap[x, y] = color;
    }

}
bitmap.Save("mandelbrot.png");

![](mandelbrot.png)